# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
print(city_data_df.shape)
city_data_df.head()

(697, 10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bengkulu,-3.8004,102.2655,23.99,86,100,4.75,ID,1672413933
1,1,wajid,3.8093,43.2471,31.30,23,5,5.78,SO,1672413933
2,2,arraial do cabo,-22.9661,-42.0278,22.98,100,100,5.14,BR,1672413933
3,3,torbay,47.6666,-52.7314,-1.90,77,75,5.66,CA,1672413934
4,4,alice springs,-23.7000,133.8833,27.75,51,75,3.09,AU,1672413934


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
#maps.configure(api_key=g_key)
# Display the map
# YOUR CODE HERE

# Configure the map plot_3
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1
   )

# Display the map plot
map_plot


ImportError: Geographic projection support requires GeoViews and cartopy.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
temp_cond1=(city_data_df["Max Temp"]<27)
temp_cond2=(city_data_df["Max Temp"]>21)
#Wind speed less than 4.5 m/s
wind_cond1 = (city_data_df["Wind Speed"]<4.5)
#Zero cloudiness
cloud_cond = (city_data_df["Cloudiness"]==0)
weather_df=city_data_df.loc[temp_cond1 & temp_cond2 & wind_cond1 & cloud_cond]
# YOUR CODE HERE
filtered_weather_df = weather_df
# Drop any rows with null values
# YOUR CODE HERE
filtered_weather_df = filtered_weather_df.dropna()
# Display sample data
# YOUR CODE HERE
filtered_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
68,68,abu dhabi,24.4667,54.3667,22.06,64,0,2.06,AE,1672413862
269,269,dwarka,22.2394,68.9678,22.44,59,0,3.92,IN,1672414009
314,314,sao filipe,14.8961,-24.4956,22.71,60,0,3.56,CV,1672414022
336,336,sur,22.5667,59.5289,22.20,54,0,3.66,OM,1672414028
375,375,makakilo city,21.3469,-158.0858,21.10,82,0,2.06,US,1672414042
403,403,pimentel,-6.8367,-79.9342,26.16,65,0,3.60,PE,1672414050
432,432,bilma,18.6853,12.9164,22.61,16,0,2.89,NE,1672414059
526,526,cabo san lucas,22.8909,-109.9124,21.40,74,0,3.60,MX,1672413864
562,562,arlit,18.7369,7.3853,26.00,9,0,1.31,NE,1672414097
587,587,ewa beach,21.3156,-158.0072,21.89,85,0,2.06,US,1672414105


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = filtered_weather_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"]=""


# Display sample data
# YOUR CODE HERE
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
68,68,abu dhabi,24.4667,54.3667,22.06,64,0,2.06,AE,1672413862,
269,269,dwarka,22.2394,68.9678,22.44,59,0,3.92,IN,1672414009,
314,314,sao filipe,14.8961,-24.4956,22.71,60,0,3.56,CV,1672414022,
336,336,sur,22.5667,59.5289,22.20,54,0,3.66,OM,1672414028,
375,375,makakilo city,21.3469,-158.0858,21.10,82,0,2.06,US,1672414042,
403,403,pimentel,-6.8367,-79.9342,26.16,65,0,3.60,PE,1672414050,
432,432,bilma,18.6853,12.9164,22.61,16,0,2.89,NE,1672414059,
526,526,cabo san lucas,22.8909,-109.9124,21.40,74,0,3.60,MX,1672413864,
562,562,arlit,18.7369,7.3853,26.00,9,0,1.31,NE,1672414097,
587,587,ewa beach,21.3156,-158.0072,21.89,85,0,2.06,US,1672414105,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    Lat = row ["Lat"]
    Lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"

    params["bias"] = f"proximity:{Lng},{Lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
dwarka - nearest hotel: The Dwarika Hotel
sao filipe - nearest hotel: Bela Vista
sur - nearest hotel: Sur Hotel
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
pimentel - nearest hotel: Hospedaje Calle del Sol
bilma - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
arlit - nearest hotel: No hotel found
ewa beach - nearest hotel: Hampton Inn & Suites Oahu/Kapolei
calama - nearest hotel: Apart Hotel Geotel Calama


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
68,68,abu dhabi,24.4667,54.3667,22.06,64,0,2.06,AE,1672413862,Marriott Downtown Abu-Dhabi
269,269,dwarka,22.2394,68.9678,22.44,59,0,3.92,IN,1672414009,The Dwarika Hotel
314,314,sao filipe,14.8961,-24.4956,22.71,60,0,3.56,CV,1672414022,Bela Vista
336,336,sur,22.5667,59.5289,22.20,54,0,3.66,OM,1672414028,Sur Hotel
375,375,makakilo city,21.3469,-158.0858,21.10,82,0,2.06,US,1672414042,Embassy Suites by Hilton Oahu Kapolei
403,403,pimentel,-6.8367,-79.9342,26.16,65,0,3.60,PE,1672414050,Hospedaje Calle del Sol
432,432,bilma,18.6853,12.9164,22.61,16,0,2.89,NE,1672414059,No hotel found
526,526,cabo san lucas,22.8909,-109.9124,21.40,74,0,3.60,MX,1672413864,Comfort Rooms
562,562,arlit,18.7369,7.3853,26.00,9,0,1.31,NE,1672414097,No hotel found
587,587,ewa beach,21.3156,-158.0072,21.89,85,0,2.06,US,1672414105,Hampton Inn & Suites Oahu/Kapolei


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)